In [174]:
from collections import defaultdict

import pandas as pd
import numpy as np

from tqdm import tqdm
import random
import json
import os

import collections

In [37]:
NEG_SAMPLE_SIZE = 100
info = {'NEG_SAMPLE_SIZE': NEG_SAMPLE_SIZE}

if not (os.path.exists('data')):
    os.mkdir('data')
with open('data//info.txt', 'w') as info_file:
            json.dump(info, info_file)

In [3]:
# df = pd.read_csv ('clear_data_realbank.csv')
# direct = pd.read_csv('mapped_poitns.csv', sep='\t', index_col=0)
# direct['RETAILER'] = direct.field_1.str.split(' ').apply(lambda x:x[-1])
# direct.drop('field_1', inplace=True, axis=1)
# direct.drop('lat', inplace=True, axis=1)
# direct.drop('lon', inplace=True, axis=1)
# direct.drop('iter', inplace=True, axis=1)
# df = pd.merge(df, direct, on = 'RETAILER', how='left')
# df['district'] = df['district'].fillna('Другое')
# df.head()
# df.to_csv(r'clear_data_realbank.csv', index=False)

In [4]:
# df = pd.read_csv('clear_data_realbank.csv')
# df.drop('TRANS_DETAIL', inplace=True, axis=1)
# df.rename(columns = {'RETAILER' : 'tail', 'CustomerKey' : 'head',
#                         'MCC' : 'tail_type', 'AMOUNT_EQ' : 'relation', 'district': 'district'  }, 
#                             inplace = True) 
# # некоторые id магазинов имели разные категории. Данным магазинам присваивается наиболее встречаемая категория среди данного магазина
# for i in tqdm(df['tail'].unique()):
#     categories = list(df[df['tail']==i].tail_type)
#     if categories != [categories[0]]*len(categories):
#         dict_categories = collections.Counter(categories)
#         max_categories = sorted(dict_categories, key=dict_categories.get, reverse=True)[0]
#         df.loc[(df['tail'] == i), 'tail_type'] = max_categories

# df.to_csv(r'data_unique_tailtype.csv', index=False)

In [5]:
# df:   USER --- BUYS --- SHOP
# df2:  SHOP --- BELONGS TO --- CATEGORY

df = pd.read_csv ('data_unique_tailtype.csv')
df = df[(df.tstmp >= '2018-01-01 00:00:00+03:00') & (df.tstmp < '2019-01-01 00:00:00+03:00')]

# удаление непопулярных категорий магазинов
top_mcc = list(df.tail_type.value_counts()[:10].rename_axis('unique_values').reset_index(name='counts')['unique_values'])
df = df[df.tail_type.isin(top_mcc)].reset_index(drop=True)
df = df.sort_values('tail_type')
df['category'] = df['tail_type']

# удаление элементов, которых не будет в тренирововчной выбрке, но будут в тестовой
df['tstmp_cond'] = df['tstmp'] < '2018-08-01 00:00:00+03:00'
temp = pd.DataFrame(df['tstmp_cond'].groupby([df['tail']]).any())
tail_del = list(temp[temp.tstmp_cond == False].index)
df = df.query("tail not in @tail_del")
temp = pd.DataFrame(df['tstmp_cond'].groupby([df['head']]).any())
head_del = list(temp[temp.tstmp_cond == False].index)
df = df.query("head not in @head_del")
df.drop('tstmp_cond', inplace=True, axis=1)

# присвоение уникальным id пользователей, магазинов и категорий   чисел от 0 до len(уникальных id) и создание таблицы реальных id и присвоенных индексов
emb_to_real = pd.DataFrame(columns = ['real_id', 'index_id', 'type'])
persons = df['head'].nunique()
shops = df['tail'].nunique()
emb_to_real['real_id'] = np.concatenate((df['head'].unique(),df['tail'].unique(),df['tail_type'].unique()))
emb_to_real['index_id'] = np.concatenate((pd.factorize(df['head'].unique())[0],(pd.factorize(df['tail'].unique())[0]+ persons),(pd.factorize(df['tail_type'].unique())[0] + persons + shops)))
emb_to_real['type'] = ['head']*len(df['head'].unique())+['tail']*len(df['tail'].unique())+['category']*len(df['tail_type'].unique())
emb_to_real.to_csv(r'data/emb_to_real.csv', index=False)
df['head'] = df['head'].map(emb_to_real.set_index('real_id')['index_id'])
df['tail'] = df['tail'].map(emb_to_real.set_index('real_id')['index_id'])
df['tail_type'] = df['tail_type'].map(emb_to_real.set_index('real_id')['index_id'])
df['amount'] = df['relation']
df['relation'] = 0

df.to_csv(r'data/df_2.csv', index=False)

# словать с типом сущности и списоком их индексов (для negative samples)
dict_id = {}
dict_id['person'] = list(range(0,max(df['head'])+1))
dict_id['tail'] = list(range(min(df['tail']),max(df['tail'])+1))
dict_id['tail_type'] = list(range(min(df['tail_type']),max(df['tail_type'])+1))

df.drop('district', inplace=True, axis=1)
df.drop('category', inplace=True, axis=1)
df.drop('amount', inplace=True, axis=1)

df2 = df.copy(deep=True)
df2['head'] = df2['tail']
df2['tail'] = df2['tail_type']
df2['tstmp'] = '2018-01-01 00:00:00+03:00'
df2['relation'] = 1
df2.drop('tail_type', inplace=True, axis=1)
df2 = df2.drop_duplicates()
df.drop('tail_type', inplace=True, axis=1)

df = df.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

# # создание негативных сущностей 
# neg_head --> tail      neg_tail --> head
df['neg_head'] = [random.sample(dict_id['person'],NEG_SAMPLE_SIZE) for i in range(len(df))]
df['neg_tail'] = [random.sample(dict_id['tail'], NEG_SAMPLE_SIZE) for i in range(len(df))]
df2['neg_head'] = [random.sample(dict_id['tail'],NEG_SAMPLE_SIZE) for i in range(len(df2))]
df2['neg_tail'] = [random.sample(dict_id['tail_type']*NEG_SAMPLE_SIZE,NEG_SAMPLE_SIZE) for i in range(len(df2))]
print(len(df)+len(df2))
df = pd.concat([df, df2], ignore_index=True)
print(len(df))
df.to_csv(r'data/df_100_2.csv', index=False)

# разбиение на выборки по времени
train = df[(df.tstmp >= '2018-01-01 00:00:00+03:00') & (df.tstmp < '2018-08-01 00:00:00+03:00')]
train.drop('tstmp', inplace=True, axis=1, errors='ignore')
valid = df[(df.tstmp >= '2018-08-01 00:00:00+03:00') & (df.tstmp < '2018-10-01 00:00:00+03:00')]
valid.drop('tstmp', inplace=True, axis=1, errors='ignore')
test = df[(df.tstmp >= '2018-10-01 00:00:00+03:00') & (df.tstmp < '2019-01-01 00:00:00+03:00')]
test.drop('tstmp', inplace=True, axis=1, errors='ignore')

/data/home/eegorova/.conda/envs/ml/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


3793035
3793035


/data/home/eegorova/.conda/envs/ml/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
train_count, train_true_head, train_true_tail = defaultdict(lambda: 4), defaultdict(list), defaultdict(list)
for i in tqdm(train.index):
    head, relation, tail = train.loc[i,'head'], train.loc[i,'relation'],  train.loc[i,'tail']
    train_count[(head, relation)] += 1
    train_count[(tail, -relation-1)] += 1
    train_true_head[(relation, tail)].append(head)
    train_true_tail[(head, relation)].append(tail)

train['subsampling_weight'] = [(1/(train_count[(train.loc[i,'head'], train.loc[i,'relation'])] 
                                + train_count[(train.loc[i,'tail'], -train.loc[i,'relation']-1)]))**(1/2)
                                                                                for i in train.index]

100%|██████████| 2145183/2145183 [01:35<00:00, 22548.46it/s]
/data/home/eegorova/.conda/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
train.to_csv(r'data/train_100_2.csv', index=False)
valid.to_csv(r'data/valid_100_2.csv', index=False)
test.to_csv(r'data/test_100_2.csv', index=False)

In [8]:
# # словарь с типами сущностей и списком их индексов
# dict_id = {}
# dict_id['person'] = list(range(0,max(df['head'])+1))
# for tail_type in df['tail_type'].unique():
#     dict_id[tail_type] = list(range(min(df[df.tail_type==tail_type]['tail']),(max(df[df.tail_type==tail_type]['tail'])+1)))
# df.to_csv(r'data/df_100_2.csv', index=False)
# df.drop('district', inplace=True, axis=1)
    
# # # создание негативных сущностей 
# # neg_head --> tail      neg_tail --> head
# df['neg_head'] = [random.sample(dict_id['person'],NEG_SAMPLE_SIZE) for _ in range(len(df))]
# df['neg_tail'] = [random.sample(dict_id[i], NEG_SAMPLE_SIZE) for i in df.tail_type]

# # # траты разбиваются на 8 категорий и в дальнейшем будут характеризировать отношения
# # df.relation = pd.qcut(df.relation, q=8, 
# # labels=["small", "medium_small", "medium_small_2", 'medium_1', 'medium_2', 'medium_large_2', 'medium_large', 'large'])    
# # # присвоение чисел отношениям
# # dict_rel = {key: idx for idx,key in enumerate(pd.factorize(df.relation)[1].categories)}
# # df['relation'] = df['relation'].apply(lambda x: dict_rel[x])
# # df['relation'] = 0
# # разбиение на выборки по времени

### For Pykeen

In [215]:
train = pd.read_csv(r'data/train_100_2.csv')
test = pd.read_csv(r'data/test_100_2.csv')
valid = pd.read_csv(r'data/valid_100_2.csv')

In [216]:
train.drop('subsampling_weight', inplace=True, axis=1, errors='ignore')
train.drop('neg_head', inplace=True, axis=1, errors='ignore')
train.drop('neg_tail', inplace=True, axis=1, errors='ignore')
test.drop('neg_head', inplace=True, axis=1, errors='ignore')
test.drop('neg_tail', inplace=True, axis=1, errors='ignore')

train = train.reindex(columns=['head', 'relation', 'tail'])
test = test.reindex(columns=['head', 'relation', 'tail'])

In [217]:
test_pos = test.groupby('head')['tail'].apply(set)
all_tail = set(range(min(train['tail']), max(train['tail'])+1))
test_neg = all_tail - test_pos

test_map_ndcg = pd.DataFrame({'head': list(test_neg.index), 'relation': 0, 
                              'tail': 4046, 'negatives': [list(i) for i in test_neg],
                              'positives': [list(i) for i in test_pos]})

test_map_ndcg['all_shop']= [sorted(x[0]+x[1]) for x in test_map_ndcg[['negatives', 'positives']].values.tolist()]

train['val']=1
freq = train.pivot_table(index='head',
                           columns='tail',
                           values='val',
                           fill_value=0, aggfunc=np.sum)
train.drop('val', inplace=True, axis=1, errors='ignore')
test_map_ndcg['scores'] = [list(freq.loc[row['head'],row['all_shop']]) for index, row in test_map_ndcg.iterrows()]

test_map_ndcg.to_csv(r'data/test_map_ndcg.csv', index=False)

In [188]:
test_unknown_link = test.copy()
train['t_h'] = train['tail'].astype(str) +'_' + train['head'].astype(str)
test_unknown_link['t_h'] = test_unknown_link['tail'].astype(str) +'_' + test_unknown_link['head'].astype(str)
test_unknown_link = test_unknown_link[~test_unknown_link['t_h'].isin(train['t_h'].unique().tolist())]
train.drop('t_h', inplace=True, axis=1, errors='ignore')
test_unknown_link.drop('t_h', inplace=True, axis=1, errors='ignore')

In [204]:
test_pos = test.groupby('head')['tail'].apply(set)
all_tail = set(range(min(train['tail']), max(train['tail'])+1))
test_neg = all_tail - test_pos
test_pos = test_unknown_link.groupby('head')['tail'].apply(set)
test_neg = test_neg[test_neg.index.isin(test_pos.index)]

test_map_ndcg_u = pd.DataFrame({'head': list(test_neg.index), 'relation': 0, 
                              'tail': 4046, 'negatives': [list(i) for i in test_neg],
                              'positives': [list(i) for i in test_pos]})

test_map_ndcg_u['all_shop']= [sorted(x[0]+x[1]) for x in test_map_ndcg_u[['negatives', 'positives']].values.tolist()]

train['val']=1
freq = train.pivot_table(index='head',
                           columns='tail',
                           values='val',
                           fill_value=0, aggfunc=np.sum)
train.drop('val', inplace=True, axis=1, errors='ignore')
test_map_ndcg_u['scores'] = [list(freq.loc[row['head'],row['all_shop']]) for index, row in test_map_ndcg_u.iterrows()]

test_map_ndcg_u.to_csv(r'data/test_map_ndcg_u.csv', index=False)

In [11]:
txt_file = open('data//pykeen//train.txt', "w") 

for row in train.values:
    row = [str(x) for x in row]
    joined_string = "\t".join(row)
    txt_file.writelines(joined_string+'\n')

txt_file.close()

In [12]:
txt_file = open('data//pykeen//test.txt', "w")

for row in test.values:
    row = [str(x) for x in row]
    joined_string = "\t".join(row)
    txt_file.writelines(joined_string+'\n')

txt_file.close()

In [120]:
txt_file = open('data//pykeen//test_neg.txt', "w")

for row in test_neg.values:
    row = [str(x) for x in row]
    joined_string = "\t".join(row)
    txt_file.writelines(joined_string+'\n')

txt_file.close()

In [11]:
txt_file = open('data//pykeen//test_unknown_link.txt', "w")

for row in test_unknown_link.values:
    row = [str(x) for x in row]
    joined_string = "\t".join(row)
    txt_file.writelines(joined_string+'\n')

txt_file.close()